# Example of Dynamics Model integration
 

In [ ]:
%load_ext autoreload
%autoreload 2
import geometry as geo
import numpy as np
from duckietown_world.world_duckietown import *

### Simulation

In [ ]:

# Model Parameters
u2 = u3 = w1 = w2 = w3 = 0 # to simplify the model
u1 = w1 = 1 # main contributor from unforced dynamics
uar = ual = war = wal = 1 # input matrix

parameters = DynamicModelParameters(u1, u2, u3, w1, w2, w3, uar, ual, war, wal)

# initial configuration
init_pose = np.array([0,0])
init_vel = np.array([0,0])
init_time = 0

q0 = geo.SE2_from_R2(init_pose)
v0 = geo.se2_from_linear_angular(init_vel, 0)
c0 = q0, v0

# starting time
t0 = 0
state = parameters.initialize(c0=c0, t0=t0)


del_t = 1

for i in range(5):
    # input = (left: -0.1, right: 0.1) must generate a counter-clockwise rotation (increasing theta)
    commands = PWMCommands(-0.1, 0.1)
    state = state.integrate(del_t, commands)
    
    qv0 = state.TSE2_from_state()
    current_p, theta = geo.translation_angle_from_SE2(qv0)
    
    print('pose: {}'.format(current_p))
    print('theta: {}'.format(np.rad2deg(theta)))

